In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon, Point
import plotly 

DATA_DIR = "../data/"

In [33]:
df_distances =  pd.read_csv(DATA_DIR+"final_distances.csv")
set(df_distances['linkid_county'])
link_id_dict = pd.read_csv(DATA_DIR+"link_id_dictionary_final.csv")

## Cascais

In [34]:
df_distances_cascais =  df_distances[df_distances['linkid_county'] == 'Cascais']

In [35]:
df_distances_cascais

,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling,key_bus_road,distance_bus_road,key_cascais,distance_cascais,key_metro,distance_metro,key_train,distance_train,key_oeiras,distance_oeiras,linkid_county
6080,537289168,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,96.314275,819525518,17.716660,101,0.379446,Carnide,22.005400,cascais,6.516727,747,11.526072,Cascais
6081,537289173,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,96.314275,819525518,17.827060,60,0.137061,Carnide,22.189836,cascais,6.405128,747,11.656533,Cascais
6082,537289176,R1 - AM Lisboa,Lisboa,Odivelas,770.514200,819525518,17.673972,101,0.273343,Carnide,21.998921,cascais,6.411777,747,11.491927,Cascais
6083,537289185,R1 - AM Lisboa,Lisboa,Odivelas,385.257100,819525518,17.669631,101,0.231235,Carnide,22.008783,cascais,6.376896,747,11.491176,Cascais
6084,537289191,R1 - AM Lisboa,Lisboa,Odivelas,385.257100,819525518,17.678418,101,0.151292,Carnide,22.044221,cascais,6.320131,747,11.507049,Cascais
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32690,1220914999,R1 - AM Lisboa,Lisboa,Oeiras,1069.197242,537321129,7.083069,6,0.289067,Carnide,13.384911,carcavelos,2.542080,867,1.125771,Cascais
32691,1220914999,R1 - AM Lisboa,Lisboa,Cascais,716.800477,537321129,7.083069,6,0.289067,Carnide,13.384911,carcavelos,2.542080,867,1.125771,Cascais
32692,1220914999,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,165.110186,537321129,7.083069,6,0.289067,Carnide,13.384911,carcavelos,2.542080,867,1.125771,Cascais
32693,1220914999,R1 - AM Lisboa,Lisboa,Odivelas,88.905485,537321129,7.083069,6,0.289067,Carnide,13.384911,carcavelos,2.542080,867,1.125771,Cascais


In [36]:
cascais_bus = pd.read_csv("../data/cascais_bus_stops.csv")

In [37]:
cascais_bus = cascais_bus[['stop_id','latitude', 'longitude']].rename(columns={'stop_id':'id'})
cascais_bus['Type'] = 'Cascais Bus'

In [38]:
df_distances_cascais_more3km = df_distances_cascais[
    (df_distances_cascais['distance_bus_road'] > 3) & \
    (df_distances_cascais['distance_cascais'] > 3) & \
    (df_distances_cascais['distance_metro'] > 3) & \
    (df_distances_cascais['distance_train'] > 3) & \
     (df_distances_cascais['distance_oeiras'] > 3)
]

In [39]:
df_distances_cascais_more3km_coords = df_distances_cascais_more3km.merge(link_id_dict, on='linkid')

In [40]:
df_distances_cascais_more3km_coords['Type'] = 'Linkids'

In [41]:
train_stations = pd.read_csv(DATA_DIR+'trains_location.csv')
trains_stations = train_stations.dropna()
train_stations = train_stations[train_stations['latitude'] != '<html']
train_stations['longitude'].replace(',','',inplace=True,regex=True)
train_stations['latitude'] = train_stations['latitude'].astype('float')
train_stations['longitude'] = train_stations['longitude'].astype('float')
train_geo = train_stations[['train_station','latitude','longitude']].rename(columns={'train_station':'id'})
train_geo['Type'] = 'Train Stations'
train_geo = train_geo.dropna()

In [42]:
df_plot = pd.concat([df_distances_cascais_more3km_coords, cascais_bus, train_geo])
df_plot = df_plot.fillna(0)

### > 3 km

In [43]:
import plotly.express as px
fig = px.scatter_mapbox(df_plot,
                    lat='latitude',
                    lon='longitude',
                    center=dict(lat=38.713704, lon=-9.482809),
                    zoom=11,
                    color='Type'
                    )
fig.update_layout(autosize=False, width=1000, height=700,mapbox_style="open-street-map")
fig.update_geos(fitbounds='locations')
fig.show()

### > 1 km


In [44]:
df_distances_cascais_more1km = df_distances_cascais[
    (df_distances_cascais['distance_bus_road'] > 1) & \
    (df_distances_cascais['distance_cascais'] > 1) & \
    (df_distances_cascais['distance_metro'] > 1) & \
    (df_distances_cascais['distance_train'] > 1) & \
     (df_distances_cascais['distance_oeiras'] > 1)
]
df_distances_cascais_more1km_coords = df_distances_cascais_more1km.merge(link_id_dict, on='linkid')
df_distances_cascais_more1km_coords['Type'] = 'Linkids'

In [45]:
df_plot = pd.concat([df_distances_cascais_more1km_coords, cascais_bus, train_geo])
df_plot = df_plot.fillna(0)

In [46]:
fig = px.scatter_mapbox(df_plot,
                    lat='latitude',
                    lon='longitude',
                    center=dict(lat=38.713704, lon=-9.482809),
                    zoom=11,
                    color='Type'
                    )
fig.update_layout(autosize=False, width=1000, height=700,mapbox_style="open-street-map")
fig.update_geos(fitbounds='locations')
fig.show()

# Oeiras

In [47]:
df_distances_oerias =  df_distances[df_distances['linkid_county'] == 'Oeiras']

In [48]:
df_distances_oeiras_more1km = df_distances_oerias[
    (df_distances_oerias['distance_bus_road'] > 0.5) & \
    (df_distances_oerias['distance_cascais'] > 0.5) & \
    (df_distances_oerias['distance_metro'] > 0.5) & \
    (df_distances_oerias['distance_train'] > 0.5) & \
     (df_distances_oerias['distance_oeiras'] > 0.5)
]
df_distances_oeiras_more1km['Type'] = 'Linkids'
df_distances_oeiras_more1km_coords = df_distances_oeiras_more1km.merge(link_id_dict, on='linkid')
df_distances_oeiras_more1km_coords['Type'] = 'Linkids'
df_distances_oeiras_more1km

<ipython-input-48-ee55f76f3fba>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling,key_bus_road,distance_bus_road,key_cascais,distance_cascais,key_metro,distance_metro,key_train,distance_train,key_oeiras,distance_oeiras,linkid_county,Type
11788,708459538,R1 - AM Lisboa,Lisboa,Oeiras,39.015695,819525518,3.229556,84,3.366199,Colégio Militar,9.512640,paco-de-arcos,2.025771,108,0.517337,Oeiras,Linkids
11794,708459562,R1 - AM Lisboa,Lisboa,Odivelas,88.905485,819525518,3.207536,84,3.391467,Colégio Militar,9.487172,paco-de-arcos,2.035276,833,0.521798,Oeiras,Linkids
11795,708459562,R1 - AM Lisboa,Lisboa,Oeiras,39.015695,819525518,3.207536,84,3.391467,Colégio Militar,9.487172,paco-de-arcos,2.035276,833,0.521798,Oeiras,Linkids
11806,708459597,R1 - AM Lisboa,Lisboa,Oeiras,39.015695,819525518,3.241452,84,3.352429,Colégio Militar,9.526760,paco-de-arcos,2.020245,108,0.507369,Oeiras,Linkids
16760,819465922,R1 - AM Lisboa,Lisboa,Oeiras,96.314275,819550880,0.718375,84,5.949441,Colégio Militar,7.685079,cruz-quebrada,1.232539,839,0.508795,Oeiras,Linkids
16761,819466752,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,177.810969,819550880,0.733711,84,5.956369,Colégio Militar,7.656812,cruz-quebrada,1.256097,838,0.519802,Oeiras,Linkids
16762,819466752,R1 - AM Lisboa,Lisboa,Lisboa,128.419033,819550880,0.733711,84,5.956369,Colégio Militar,7.656812,cruz-quebrada,1.256097,838,0.519802,Oeiras,Linkids
19201,864151610,R1 - AM Lisboa,Lisboa,Sintra,26.528318,819525518,5.798834,84,1.093228,Carnide,11.438345,paco-de-arcos,3.523892,199,0.513183,Oeiras,Linkids
19618,877282436,R1 - AM Lisboa,Lisboa,Oeiras,233.203075,537321129,5.470869,1,1.418346,Colégio Militar,12.207353,oeiras,1.842579,826,0.571002,Oeiras,Linkids
19794,877361332,R1 - AM Lisboa,Lisboa,Oeiras,30.655189,819550880,0.518354,84,6.241614,Colégio Militar,7.488750,cruz-quebrada,1.095458,459,0.517056,Oeiras,Linkids


In [49]:
oeiras_bus = pd.read_csv("../data/oeiras_bus_stops.csv")
oeiras_bus = oeiras_bus.drop('Conditions', axis=1).rename(columns = {"stop_id":"id"})
oeiras_bus['Type'] = 'Oeiras Bus'

In [50]:
df_plot = pd.concat([df_distances_oeiras_more1km_coords, oeiras_bus, train_geo])
df_plot = df_plot.fillna(0)

In [51]:
fig = px.scatter_mapbox(df_plot,
                    lat='latitude',
                    lon='longitude',
                    center=dict(lat=38.7195, lon=-9.2631),
                    zoom=11,
                    color='Type'
                    )
fig.update_layout(autosize=False, width=1000, height=700,mapbox_style="open-street-map")
fig.update_geos(fitbounds='locations')
fig.show()

# Lisboa

In [52]:
geo_linkid =  pd.read_csv(DATA_DIR+"link_id_dictionary_final.csv")
data_bus_routes = pd.read_csv(DATA_DIR+"BusRoutes.txt", sep="|", encoding='latin1')
geo_linkid = geo_linkid[geo_linkid['linkid'].isin(data_bus_routes['linkid'])]

In [53]:
data_bus_routes = data_bus_routes[data_bus_routes['linkid'].isin(geo_linkid['linkid'])]

In [54]:
geo_linkid = geo_linkid.set_index('linkid')
geo_dict = geo_linkid[['latitude','longitude']].to_dict('index')

In [55]:
data_bus_routes['latitude'] = data_bus_routes['linkid'].apply(lambda x: geo_dict[x]['latitude'])
data_bus_routes['longitude'] = data_bus_routes['linkid'].apply(lambda x: geo_dict[x]['longitude'])
geometry = [Point(xy) for xy in zip(data_bus_routes['latitude'], data_bus_routes['longitude'])]

In [56]:
geodf = gpd.GeoDataFrame(data_bus_routes, geometry=geometry)
geodf['Type'] = 'Bus'
bus_geo = geodf[['IDRoute','latitude','longitude','Type']].rename(columns={'IDRoute':'id'})

In [57]:
metro_stations = pd.read_csv(DATA_DIR+'metro_location.csv')
metro_stations = metro_stations.fillna('Unknown')
metro_stations['Type'] = 'Metro'
metro_geo = metro_stations[['station_name','latitude','longitude','Type']].rename(columns={'station_name':'id'})

In [58]:
df_distances_lisboa =  df_distances[df_distances['linkid_county'] == 'Lisboa']
df_distances_lisboa_more1km = df_distances_lisboa[
    (df_distances_lisboa['distance_bus_road'] > 0.5) & \
    (df_distances_lisboa['distance_cascais'] > 0.5) & \
    (df_distances_lisboa['distance_metro'] > 0.5) & \
    (df_distances_lisboa['distance_train'] > 0.5) & \
     (df_distances_lisboa['distance_oeiras'] > 0.5)
]
df_distances_lisboa_more1km['Type'] = 'Linkids'
df_distances_lisboa_more1km_coords = df_distances_lisboa_more1km.merge(link_id_dict, on='linkid')
df_distances_lisboa_more1km_coords['Type'] = 'Linkids'

<ipython-input-58-f529020b654e>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [59]:
df_plot = pd.concat([df_distances_lisboa_more1km_coords])
df_plot = df_plot.fillna(0)

### > 0.5km

In [60]:
fig = px.scatter_mapbox(df_plot,
                    lat='latitude',
                    lon='longitude',
                    center=dict(lat=38.7841, lon=-9.1470),
                    zoom=11,
                    color='Type'
                    )
fig.update_layout(autosize=False, width=1000, height=700,mapbox_style="open-street-map")
fig.update_geos(fitbounds='locations')
fig.show()

In [61]:
df_plot = pd.concat([df_distances_lisboa_more1km_coords, train_geo, bus_geo, metro_geo])
df_plot = df_plot.fillna(0)

In [62]:
fig = px.scatter_mapbox(df_plot,
                    lat='latitude',
                    lon='longitude',
                    center=dict(lat=38.7841, lon=-9.1470),
                    zoom=11,
                    color='Type'
                    )
fig.update_layout(autosize=False, width=1000, height=700,mapbox_style="open-street-map")
fig.update_geos(fitbounds='locations')
fig.show()